### 1. Importing Librabries

In [59]:
import os

import numpy as np

import pandas as pd

from sklearn.model_selection import train_test_split

### 2. Reading the Data

In [60]:
PROJECT_DIR = r"C:\Users\harsh\OneDrive\Desktop\Flight-Price-Prediction"
DATA_DIR = 'data'

In [61]:
def read_data(name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR,DATA_DIR,file_name)
    return pd.read_csv(file_path)

In [27]:
flights = read_data("flight_price")

In [62]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- The dataset contains 10,683 rows and 11 features
- Columns 'Route' and 'Total_Stops' have missing value each
- The data types of some features isn't approriate

### 3. Preliminary Analysis

### 3.1 Check Data Types

In [28]:
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24-03-2019,Banglore,New Delhi,BLR → DEL,22:20,22-03-2024 01:10,2h 50m,non-stop,No info,3897
1,Air India,01-05-2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,09-06-2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,10-06-2024 04:25,19h,2 stops,No info,13882
3,IndiGo,12-05-2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01-03-2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [29]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

### 3.2 Check for Duplicates

In [30]:
(
    flights
    .loc[flights.duplicated(keep = False)]
    .sort_values(['Airline','Date_of_Journey','Source','Destination'])
    
)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01-03-2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,02-03-2024 23:55,39h 5m,2 stops,No info,17135
9848,Air India,01-03-2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,02-03-2024 23:55,39h 5m,2 stops,No info,17135
1495,Air India,01-04-2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,02-04-2024 01:20,15h 20m,2 stops,No info,10408
9913,Air India,01-04-2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,02-04-2024 01:20,15h 20m,2 stops,No info,10408
3598,Air India,01-05-2019,Kolkata,Banglore,CCU → GAU → DEL → BLR,09:50,02-05-2024 08:55,23h 5m,2 stops,No info,13227
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24-03-2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24-03-2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24-03-2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24-03-2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


### 3.3 Observations

- The type of 'Date_of_Journey', 'Dep_Time' and 'Arrival_Time' should be changed to datetime
- The type of 'Duration' and 'Total_stops' is mixed. It should be numeric type
- There are 220 duplicates. These should be removed

### 4, Detailed Analysis

## Airline

In [102]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [103]:
flights.Airline.str.replace('Premium economy','')

0             IndiGo
1          Air India
2        Jet Airways
3             IndiGo
4             IndiGo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: object

In [108]:
(
    flights
    .Airline
    .str.replace('Premium economy','')
    .str.replace('Business','')
    .str.title()
    .unique()
)

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Goair', 'Vistara', 'Air Asia', 'Vistara ',
       'Jet Airways ', 'Multiple Carriers ', 'Trujet'], dtype=object)

- Some of the entries have inconsistent/inaccurate values

## Date of Journey

In [114]:
pd.to_datetime(flights.Date_of_Journey,dayfirst = True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

## Source

In [117]:
flights.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

### Destination

In [119]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

### Dep_time

In [132]:
(
    flights
    .Dep_Time
    .loc[lambda ser : ser.str.contains("[^0-9:]")]
)

Series([], Name: Dep_Time, dtype: object)

In [134]:
pd.to_datetime(flights.Dep_Time).dt.time

C:\Users\harsh\AppData\Local\Temp\ipykernel_65664\1056054517.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(flights.Dep_Time).dt.time


0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

In [152]:
(
    flights
    .Arrival_Time
    .loc[lambda ser : ser.str.contains("[^0-9:]")]
    .str.split(" ")
    .str.get(1)
    .unique()
)

array(['01:10', '04:25', '10:25', '05:05', '09:20', '19:15', '12:35',
       '13:20', '08:15', '03:35', '19:00', '11:05', '10:55', '00:15',
       '02:45', '12:00', '01:30', '16:45', '09:25', '04:40', '11:25',
       '14:25', '14:35', '09:05', '01:35', '10:05', '11:10', '07:45',
       '00:55', '23:35', '00:40', '18:50', '07:55', '18:15', '13:15',
       '09:45', '06:50', '15:15', '07:40', '02:20', '18:30', '19:45',
       '19:50', '01:00', '00:05', '16:20', '00:30', '00:50', '13:45',
       '09:15', '07:10', '08:55', '04:45', '05:35', '09:00', '10:45',
       '22:10', '22:40', '05:25', '02:15', '20:20', '06:35', '01:20',
       '19:10', '10:50', '23:25', '12:15', '10:40', '23:15', '22:55',
       '09:10', '18:10', '00:25', '20:45', '21:05', '23:00', '00:45',
       '04:35', '18:00', '12:30', '02:10', '00:20', '23:20', '12:25',
       '22:35', '20:25', '00:35', '16:50', '09:30', '12:05', '10:35',
       '15:05', '18:40', '09:35', '08:50', '16:40', '19:40', '16:10',
       '21:20', '07:

In [176]:
(
    flights
    .Arrival_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
    .str.split(" ")
    .str.get(1),
    pd.to_datetime(flights.Arrival_Time,format='mixed').dt.time
)

(0        01:10
 2        04:25
 6        10:25
 7        05:05
 8        10:25
          ...  
 10666    19:00
 10667    20:20
 10672    19:00
 10673    04:25
 10674    21:20
 Name: Arrival_Time, Length: 4335, dtype: object,
 0        01:10:00
 1        13:15:00
 2        04:25:00
 3        23:30:00
 4        21:35:00
            ...   
 10678    22:25:00
 10679    23:20:00
 10680    11:20:00
 10681    14:10:00
 10682    19:15:00
 Name: Arrival_Time, Length: 10683, dtype: object)

In [228]:
flights

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24-03-2019,Banglore,New Delhi,BLR → DEL,22:20,22-03-2024 01:10,2h 50m,non-stop,No info,3897
1,Air India,01-05-2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,09-06-2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,10-06-2024 04:25,19h,2 stops,No info,13882
3,IndiGo,12-05-2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01-03-2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,09-04-2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27-04-2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27-04-2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01-03-2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648


## Duration

In [230]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains('h')]
    #.str.split(n = 1,expand = True)
    #.set_axis(['hour','minute'],axis = 'columns')
    
    
    #.isna().sum()
)

6474    5m
Name: Duration, dtype: object

In [256]:
(
    flights
    .Duration
    .drop(index = [6474])
    .str.split(expand = True)
    .set_axis(['hour','minute'],axis = 1)
    .assign(

        hour = lambda df_: (

            df_
            .hour.str.replace('h','')
            .astype(int)
            .mul(60)
        ),
        minute = lambda df_ : (

            df_
            .minute.str.replace('m','')
            .fillna('0')
            .astype(int)
            
        )
    )
    .sum(axis = 1)
    .rename('duration_minutes')
    .to_frame()
)

,duration_minutes
0,170
1,445
2,1140
3,325
4,285
...,...
10678,150
10679,155
10680,180
10681,160


In [221]:
flights.iloc[6474]

Airline                        Air India
Date_of_Journey               06-03-2019
Source                            Mumbai
Destination                    Hyderabad
Route              BOM → GOI → PNQ → HYD
Dep_Time                           16:50
Arrival_Time                       16:55
Duration                              5m
Total_Stops                      2 stops
Additional_Info                  No info
Price                              17327
Name: 6474, dtype: object

## Total Stops

In [262]:
flights.Total_Stops.isna().sum()

1

In [267]:
(
    flights
    .Total_Stops
    .replace('non-stop','0')
    .str.replace('stops?','',regex = True)
    .pipe(lambda ser : pd.to_numeric(ser))
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

### 5. Cleaning Operations

In [257]:
def convert_to_minutes(ser):
    return (

        ser
        .str.split(expand = True)
        .set_axis(['hour','minute'],axis = 1)
        .assign(

                  hour = lambda df_: (

            df_
            .hour.str.replace('h','')
            .astype(int)
            .mul(60)
        ),
        minute = lambda df_ : (

            df_
            .minute.str.replace('m','')
            .fillna('0')
            .astype(int)
            
        )
    )
    .sum(axis = 1)

        
    )

In [273]:
def clean_data(df):
    return (
        df
        .drop(index = [6474])
        .drop_duplicates()
        .assign(**{
             col : df[col].str.strip()
            for col in df.select_dtypes(include = 'O').columns           
        })
        
        .rename(columns = str.lower)

        .assign(

            airline = lambda df_: (

                df_
                .airline
                .str.replace('Premium economy','')
                .str.replace('Business','')
                .str.title()
            ),
            date_of_journey = lambda df_ : pd.to_datetime(df_.date_of_journey,dayfirst = True),
            dep_time = lambda df_ : pd.to_datetime(df_.dep_time).dt.time,
            arrival_time = lambda df_ : pd.to_datetime(df_.arrival_time,format = 'mixed').dt.time,
            duration = lambda df_ : df_.duration.pipe(convert_to_minutes),
            total_stops = lambda df_ : (

                df_
                .total_stops
                .replace('non-stop','0')
                .str.replace('stops?','',regex = True)
                .pipe(lambda ser : pd.to_numeric(ser))
            )
        )
        .drop('route',axis = 1)
    
    
    
    
    )

In [274]:
import warnings
warnings.filterwarnings('ignore')
flights_cleaned = clean_data(flights)
flights_cleaned

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0.0,No info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0.0,No info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0.0,No info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0.0,No info,12648


### 6. Split the Data

In [275]:
flights_final = flights_cleaned.sample(1000)

In [276]:
X = flights_final.drop('price',axis = 1)
Y = flights_final.price.copy()

In [279]:
X_,X_test,Y_,Y_test = train_test_split(X,Y,test_size = 0.2,random_state = 42)
X_train,X_val,Y_train,Y_val = train_test_split(X_,Y_,test_size = 0.2,random_state = 42)

print(X_train.shape,Y_train.shape)
print(X_val.shape,Y_val.shape)
print(X_test.shape,Y_test.shape)

(640, 9) (640,)
(160, 9) (160,)
(200, 9) (200,)


### 7. Export the Subset

In [280]:
def export_data(X,Y,name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR,DATA_DIR,file_name)
    X.join(Y).to_csv(file_path,index = False)
    return pd.read_csv(file_path).head()

In [281]:
export_data(X_train,Y_train,'train')

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-06-12,Kolkata,Banglore,18:55:00,18:15:00,1400,1.0,In-flight meal not included,8529
1,Indigo,2019-03-18,Mumbai,Hyderabad,06:20:00,07:45:00,85,0.0,No info,3342
2,Jet Airways,2019-06-09,Kolkata,Banglore,09:35:00,22:05:00,750,1.0,In-flight meal not included,13994
3,Jet Airways,2019-06-06,Kolkata,Banglore,20:25:00,09:45:00,800,1.0,In-flight meal not included,10539
4,Indigo,2019-03-24,Banglore,New Delhi,08:30:00,11:30:00,180,0.0,No info,7303


In [282]:
export_data(X_val,Y_val,'val')

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Air India,2019-04-24,Banglore,Delhi,21:05:00,23:55:00,170,0.0,No info,5228
1,Spicejet,2019-06-06,Delhi,Cochin,17:55:00,22:50:00,295,1.0,No info,5409
2,Multiple Carriers,2019-06-15,Delhi,Cochin,11:30:00,21:00:00,570,1.0,No info,13587
3,Jet Airways,2019-05-27,Delhi,Cochin,14:00:00,04:25:00,865,1.0,In-flight meal not included,12898
4,Indigo,2019-05-21,Delhi,Cochin,06:40:00,21:00:00,860,1.0,No info,7006


In [283]:
export_data(X_test,Y_test,'test')

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-05-18,Delhi,Cochin,08:35:00,22:30:00,835,1.0,No info,7144
1,Spicejet,2019-06-21,Mumbai,Hyderabad,22:45:00,00:15:00,90,0.0,No info,2017
2,Spicejet,2019-03-15,Mumbai,Hyderabad,13:10:00,14:30:00,80,0.0,No check-in baggage included,2385
3,Jet Airways,2019-06-27,Delhi,Cochin,19:30:00,12:35:00,1025,2.0,No info,13764
4,Spicejet,2019-05-21,Banglore,Delhi,05:55:00,08:35:00,160,0.0,No info,3311
